In [2]:
import math

t_imaging = 25 # minutes

R_t_i = 1.443 * (110/t_imaging) * (1-math.exp(-0.693*t_imaging/110))
print(f"R_t_i: {R_t_i}")

def PET_uptake_dose(patients,activity,distance):

    dose = 0.092 * patients * activity * 1 * 0.83/distance**2
    return dose # uSv/wk

def PET_imaging_dose(patients,activity,distance):

    dose = 0.092 * patients * activity * 0.85 * 0.68 * 0.41666666666 * R_t_i/distance**2
    return dose # uSv/wk

def convert_to_meters(foot,inches):
    m = (foot * 12 * 2.54 + inches * 2.54) * 10e-3
    return m

patients = 30
a = 370 # Mbq

def find_Ksec(d):
    K_sec = (91.76599999999999/d**2) * 1e3 # converts to uGy/wk
    return K_sec # uGy/wk

R_t_i: 0.925225655037042


In [15]:
d_uptake1_pt5 = convert_to_meters(7,3.84561)
print(f"d_uptake1_pt5: {d_uptake1_pt5} m")
d_uptake2_pt5 = convert_to_meters(16,6.89352)
print(f"d_uptake2_pt5: {d_uptake2_pt5} m")


d_pet_pt5 = convert_to_meters(24,9.58012)
print(f"d_pet_pt5: {d_pet_pt5} m")
print("This distance is the same for both PET and CT contribution from the imaging room")

d_uptake1_pt5: 2.231278494 m
d_uptake2_pt5: 5.051895408 m
d_pet_pt5: 7.558535048 m
This distance is the same for both PET and CT contribution from the imaging room


In [26]:
B_wallA = 2.47e-2 # Transmission factor through Wall A using 1" Pb
B_sharedwall = 4.522e-1 # Transmission factor through the shared wall using 1/4" Pb
print(f"B_wallA: {B_wallA}")
print(f"B_sharedwall: {B_sharedwall}")
print('---------------------------------------------------------')
PET_uptake1 = PET_uptake_dose(patients,a,d_uptake1_pt5)
print(f"Shielded dose from the close uptake room: {PET_uptake1 * B_wallA} uSv/wk")

PET_uptake2 = PET_uptake_dose(patients,a,d_uptake2_pt5)
print(f"Shielded dose from the far uptake room: {PET_uptake2 * B_sharedwall * B_wallA} uSv/wk")

B_wallA: 0.0247
B_sharedwall: 0.4522
---------------------------------------------------------
Shielded dose from the close uptake room: 4.205119022669736 uSv/wk
Shielded dose from the far uptake room: 0.3709434515527639 uSv/wk


In [24]:
K_sec = find_Ksec(d_pet_pt5) * 2.8e-2
print(f"Shielded K_sec at Point 5: {K_sec} uSv/wk") # 2.8e-2 transmission factor for 1/32" Pb From Figure A.2 NCRP 147

PET_imaging = PET_imaging_dose(patients,a,d_pet_pt5) * 0.9 # 0.9 is the transmission factor for the imaging room wall from Figure 1 TG 108
print(f"Shielded dose due to PET from the imaging room: {PET_imaging} uSv/wk")

Shielded K_sec at Point 5: 44.97431636725859 uSv/wk
Shielded dose due to PET from the imaging room: 3.5846133383049423 uSv/wk


In [25]:
total = PET_uptake1 * B_wallA + PET_uptake2 * B_sharedwall*B_wallA + PET_imaging + K_sec * 2.8e-2
print(f"Total dose from all room to Point 5: {total} uSv/wk")

Total dose from all room to Point 5: 9.419956670810683 uSv/wk
